# Exercise 2: Loading the Case Study Data in a Jupyter Notebook

# First Jupyter notebook
Welcome to your first jupyter notebook! The first thing to know about Jupyter notebooks is that there are two kinds of cells. This is a markdown cell.

There are a lot of different ways to mark up the text in markdown cells, including __bold__ and *italics*.

The next one will be a `code` cell.

In [ ]:
import pandas as pd

Load data

In [ ]:
df = pd.read_excel('../Data/default_of_credit_card_clients__courseware_version_1_21_19.xls')

In [ ]:
df.shape

In [ ]:
a = 5

In [ ]:
a

# Exercise 3: Verifying Basic Data Integrity

The data consist of one month of credit card account information, with historical data looking back six months. A basic quality check is to make sure that we have data for as many accounts as we think we do. In particular, account IDs should all be distinct. We can check to see the number of distinct ID's with the Pandas function `.nunique()`.

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['ID'].nunique()

Has less than the number of rows, so there are some duplicates somewhere.

In [ ]:
id_counts = df['ID'].value_counts()
id_counts.head()

In [ ]:
id_counts.value_counts()

# Boolean masks

In [ ]:
import numpy as np

In [ ]:
np.random.seed(seed=24)
random_integers = np.random.randint(low=1,high=5,size=100)

In [ ]:
random_integers[:5]

In [ ]:
is_equal_to_3 = random_integers == 3

In [ ]:
is_equal_to_3[:5]

In [ ]:
sum(is_equal_to_3)

In [ ]:
random_integers[is_equal_to_3]

# Exercise 4: Continuing Verification of Data Integrity

In [ ]:
dupe_mask = id_counts == 2

In [ ]:
dupe_mask[0:5]

In [ ]:
id_counts.index[0:5]

In [ ]:
dupe_ids = id_counts.index[dupe_mask]

In [ ]:
dupe_ids = list(dupe_ids)
len(dupe_ids)

In [ ]:
dupe_ids[0:5]

In [ ]:
df.loc[df['ID'].isin(dupe_ids[0:3]),:].head(10)

We can see some duplicates here, and it looks like every duplicate ID has one row with data, and another row with all zeros. Is this the case for every duplicate ID? Let's check.

In [ ]:
df.shape

In [ ]:
df_zero_mask = df == 0

In [ ]:
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)

In [ ]:
sum(feature_zero_mask)

It looks like there are at least as many "zero rows" as there are duplicate IDs. Let's remove all the rows with all zero features and response, and see if that gets rid of the duplicate IDs.

In [ ]:
df_clean_1 = df.loc[~feature_zero_mask,:].copy()

In [ ]:
df_clean_1.shape

In [ ]:
df_clean_1['ID'].nunique()

Looks like this worked.

# Exercise 5: Exploring and Cleaning the Data

In [ ]:
df_clean_1.info()

In [ ]:
df_clean_1.head()

In [ ]:
df_clean_1['PAY_1'].head(5)

In [ ]:
df_clean_1['PAY_1'].value_counts()

Let's throw out these missing values, which were initially hidden from us in the `.info()` output, now.

In [ ]:
valid_pay_1_mask = df_clean_1['PAY_1'] != 'Not available'

In [ ]:
valid_pay_1_mask[0:5]

In [ ]:
sum(valid_pay_1_mask)

In [ ]:
df_clean_2 = df_clean_1.loc[valid_pay_1_mask,:].copy()

In [ ]:
df_clean_2.shape

In [ ]:
df_clean_2['PAY_1'].value_counts()

In [ ]:
df_clean_2['PAY_1'] = df_clean_2['PAY_1'].astype('int64')

In [ ]:
df_clean_2[['PAY_1', 'PAY_2']].info()

# Exercise 6: Exploring the Credit Limit and Demographic Features

In [ ]:
import matplotlib.pyplot as plt #import plotting package

#render plotting automatically
%matplotlib inline

import matplotlib as mpl #additional plotting functionality

mpl.rcParams['figure.dpi'] = 400 #high resolution figures

In [ ]:
df_clean_2[['LIMIT_BAL', 'AGE']].hist()

In [ ]:
df_clean_2[['LIMIT_BAL', 'AGE']].describe()

In [ ]:
df_clean_2['EDUCATION'].value_counts()

"Education (1 = graduate school; 2 = university; 3 = high school; 4 = others)"

Assign unknown categories to other.

In [ ]:
df_clean_2['EDUCATION'].replace(to_replace=[0, 5, 6], value=4, inplace=True)

In [ ]:
df_clean_2['EDUCATION'].value_counts()

Examine and clean marriage feature as well:

In [ ]:
df_clean_2['MARRIAGE'].value_counts()

In [ ]:
#Should only be (1 = married; 2 = single; 3 = others).
df_clean_2['MARRIAGE'].replace(to_replace=0, value=3, inplace=True)

In [ ]:
df_clean_2['MARRIAGE'].value_counts()

# Deep dive: categorical features

In [ ]:
from scipy import stats
import numpy as np

Generate some synthetic near-linear data:

In [ ]:
X = np.array(range(1,11))

In [ ]:
X

In [ ]:
np.random.seed(seed=24)
Y = 2*X + np.random.normal(0, 1, 10)

In [ ]:
Y

In [ ]:
slope_line, intercept_line, r_value, p_value, std_err = stats.linregress(X,Y)

In [ ]:
mpl.rcParams['figure.dpi'] = 400
mpl.rcParams['font.size'] = 16
plt.plot(X, intercept_line + slope_line*X, 'r-')
plt.bar(X,Y, align='center')
plt.xlabel('Self-reported customer satisfaction on scale from 1-10')
plt.ylabel('Average time spent on\nwebsite, example 1 (minutes)')
plt.yticks(range(0,25,5))

And some synthetic polynomial data

In [ ]:
np.random.seed(seed=20)
noise = np.random.normal(0, 1, 10)
Y_poly = -1*(X-2)*(X-9) + 10 + noise

In [ ]:
slope_poly, intercept_poly, r_value, p_value, std_err = stats.linregress(X,Y_poly)

In [ ]:
plt.plot(X, intercept_poly + slope_poly*X, 'r-')
plt.bar(X,Y_poly,align='center')
plt.xlabel('Self-reported customer satisfaction on scale from 1-10')
plt.ylabel('Average time spent on\nwebsite, example 2 (minutes)')

In [ ]:
df_clean_2.groupby('EDUCATION').agg({'default payment next month':'mean'})

In [ ]:
df_clean_2.groupby('EDUCATION').agg({'default payment next month':'mean'}).plot.bar(legend=False)
plt.ylabel('Default rate')
plt.xlabel('Education level: ordinal encoding')

# Exercise 7: Implementing One-Hot Encoding for a Categorical Feature

In [ ]:
df_clean_2['EDUCATION'].head()

Make a new column of categorical strings, using the data dictionary

In [ ]:
df_clean_2['EDUCATION_CAT'] = 'none'

In [ ]:
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head(10)

"Education (1 = graduate school; 2 = university; 3 = high school; 4 = others)"

In [ ]:
cat_mapping = {
    1: "graduate school",
    2: "university",
    3: "high school",
    4: "others"
}

In [ ]:
df_clean_2['EDUCATION_CAT'] = df_clean_2['EDUCATION'].map(cat_mapping)

In [ ]:
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head(10)

In [ ]:
edu_ohe = pd.get_dummies(df_clean_2['EDUCATION_CAT'])
edu_ohe.head(10)

In [ ]:
df_with_ohe = pd.concat([df_clean_2, edu_ohe], axis=1)
df_with_ohe[['EDUCATION_CAT', 'graduate school',
             'high school', 'university', 'others']].head(10)

Save the fruits of your exploratory labors

In [ ]:
# df_with_ohe.to_csv('../Data/Chapter_1_cleaned_data.csv', index=False)

# Exploring the Financial History Features in the Dataset

In [ ]:
df = pd.read_csv('../Data/Chapter_1_cleaned_data.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
pay_feats = ['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

In [ ]:
df[pay_feats].describe()

In [ ]:
df[pay_feats[0]].value_counts().sort_index()

In [ ]:
pay_feats[0]

In [ ]:
mpl.rcParams['figure.dpi'] = 400
mpl.rcParams['font.size'] = 12
df[pay_feats[0]].hist()

In [ ]:
pay_1_bins = np.array(range(-2,10)) - 0.5
pay_1_bins

In [ ]:
df[pay_feats[0]].hist(bins=pay_1_bins)
plt.xlabel('PAY_1')
plt.ylabel('Number of accounts')

In [ ]:
for feat in pay_feats:
    print(df[feat].value_counts().sort_index())

In [ ]:
mpl.rcParams['font.size'] = 4
df[pay_feats].hist(bins=pay_1_bins, layout=(2,3))

In [ ]:
df.loc[df['PAY_2']==2, ['PAY_2', 'PAY_3']].head()

# Activity 1: Exploring Remaining Financial Features in the Dataset

In [ ]:
bill_feats = ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']
pay_amt_feats = ['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']

In [ ]:
df[bill_feats].describe()

In [ ]:
df[bill_feats].hist(bins=20, layout=(2,3))

In [ ]:
df[pay_amt_feats].describe()

In [ ]:
df[pay_amt_feats].hist(layout=(2,3), xrot=30)

In [ ]:
pay_zero_mask = df[pay_amt_feats] == 0

In [ ]:
pay_zero_mask.head()

In [ ]:
pay_zero_mask.sum()

In [ ]:
df[pay_amt_feats][~pay_zero_mask].apply(np.log10).hist(layout=(2,3))